<a href="https://colab.research.google.com/github/mojtabaSefidi/AI-Challenge-PoliceVsThieves/blob/main/Event_Detection_Simulator_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


### Import & Installation

In [ ]:
from collections import Counter, OrderedDict
import operator
import pandas as pd
import numpy as np
import itertools
import random
import re
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
import pickle
import time

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !pip install keybert[all]
# from keybert import KeyBERT
# kw_model = KeyBERT()

In [ ]:
!pip install hazm
from hazm import *
from hazm import stopwords_list
!gdown 1LDlffCuIatgoCKlxGInTh8W3b1fYgJ_y
!unzip resources.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1LDlffCuIatgoCKlxGInTh8W3b1fYgJ_y 

unzip:  cannot find or open resources.zip, resources.zip.zip or resources.zip.ZIP.


In [ ]:
Pars_Bert_Model = SentenceTransformer('HooshvareLab/bert-base-parsbert-uncased',device='cuda')

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Define Positive , Negative & Spam Key-Phrases

### Stop Words & Spam Key Phrase

In [ ]:
stop_words = set(['زیرا','از','به','در','چه','پس','ولی','چون','یا','نیز','را','و', 'هم' , 'اگر' ,'تا' ,'اما' , 'که', 'با', 'اگرچه', 'آن', 'این', 'اکنون','همین','همان','چنانچه','آ‌نگاه','چنانچه','چنانچه','بر','لایک', 'برای','است','آیا'])

spam_keyphrase = ['پادکست','نظرسنجی','تلگرام','ترکیب']

### Sport Key-Phrases

In [ ]:
list_of_sport_positive_keyphrase = [
                           ['مدال','طلا'],['مدال','برنز'],['مدال','نقره'],
                           ['مسابقه','فینال'],['تیم','قهرمانی','مسابقات'],
                           ['ورزشکار','محرومیت'],['تیم','قهرمانی','جام'],
                           ['دوپینگ','تست','مثبت'],['تیم','قهرمان','مسابقات'],
                           ['بازیکن','توافق','تیم'],['ساله','قرارداد','توافق'],['امضای','قرارداد'],
                           ['فسخ','قرارداد','کرد'],['فسخ','قرارداد','شد'],['قرارداد','تمدید','کرد'],
                           ['انتقالات','نقل','بمب','توافق'],['انتقالات','نقل','بمب','پیوست'],
                           ['قرضی','قرارداد'],['جایزه','بهترین'],['برگزاری','مسابقات'],
                           ['لغو','مسابقات'],['تعویق','مسابقات'],['برگزار','مسابقات'],
                           ['رکورد','گینس'],['رکورد','المپیک','شکسته'],['رکورد','جهان','شکسته'],
                           ['رکورد','آسیا','شکسته'],['رکورد','المپیک','شکست'],
                           ['رکورد','جهان','شکست'],['رکورد','آسیا','شکست'],
                           ['رکورد','شکسته','شد'],['رکورد','به','حمله'],['در','یک','قدمی'],
                           ['پناهجو','ورزشکار'],['پناهنده'],['هاله','از','ابهام'],['همکاری','قطع'],
                           ['اخراج','سرمربی'],['استعفا','سرمربی'],['شد','سرمربی','موقت'],
                           ['بار','اولین','برای'],['ستاره','درخشش'],['باورنکردنی'],
                           ['بانوان','استادیوم'],['مصدومیت','شدید'],['مسابقات','صعود'],['کرد','سقوط'],
                           ['رقابت','صعود'],['تست','کرونای','مثبت'],['کسب','سهمیه'],
                           ['تست','کرونا','مثبت'],['کمیته','انضباطی','رای'],['کمیته','انضباطی','اخضار'],
                           ['محرومیت','انضباطی'],['شعار','هواداران'],['اعتراض','هواداران'],
                           ['اعتراض','شدید'],['محرومیت','ساله'],['پرونده','جریمه'],['شد','قطعی'],
                           ['خداحافظی','هواداران'],['خداحافظی','تیم'],['وداع','تیم'],['در','قدمی','یک'],
                           ['فوت','کرد'],['فوت','شد'],['در','جدایی','آستانه'],['گل','آقای','شد'],
                           ['پزشکی','تست'],['راهی','بیمارستان'],['شد','بیهوش'],['تیغ','جراحی'],['تیغ','جراحان'],
                           ['شد','رونمایی'],['کرد','رونمایی'],['میلیون','طلب'],['منصوب','شد'],
                           ['طلا','توپ','برنده'],['طلا','کفش','برنده'],['هواداران','هجوم'],
                           ['میادین','بازگشت','به'],['میادین','دوری','از'],['تاریخی','رکورد'],
                           ['لو','رفت'],['مشخص','شد','تاریخ'],['مشخص','شد','قیمت'],['شد','تصویب'],
                           ['مشخص','شد','جانشین'],['مشخص','شد','سرمربی'],['مشخص','شد','لیست'],['مشخص','شد','زمان'],
                           ['مشخص','شد','قهرمان'],['مشخص','شدند','نفرات','برتر'],['مشخص','شد','نتایج'],['ناباورانه'],
                           ['شدید','اعتراض'],['شدید','انتقاد'],['فوری'],['کشور','ثالث','بازی'],['شدید','درگیری'],
                           ['قلبی','ایست'],
                                 
]

### Technology Key-Phrases

In [ ]:
list_of_technology_positive_keyphrase = [
                           ['جدید','منتشر','نسخه'],['رونمایی','نسخه'],
                           ['سایبری','حمله'],['هک','شد'],['حمله','هکر'],
                           ['حمله','هکرها'],['اطلاعات','افشای'],['لو','رفت'],
                           ['لو','رفتن'],['برگزار','رویداد'],['بار','اولین','برای'],
                           ['ماهواره'],['نوآوری','جدید'],['رونمایی','شد'],
                           ['رونمایی','کرد'],['تاریخ','عرضه'],['معرفی','شد'],
                           ['غول','فناوری'],['گروه','هکری'],['آپدیت','بزرگ'],
                           ['حفره','امنیتی'],['آپدیت','جدید'],['اختلال'],['باگ','امنیتی'],
                           ['نفوذ','سایبری'],['پشتیبانی','میکند'],['عرضه','شد'],['عرضه','میشود'],
                           ['جدید','سری'],['ارز','نیمایی'],['خرید','شرکت'],['کرد','شکایت'],
                           ['سرویس','فعال'],['سرویس','جدید'],['قابلیت','جدید'],['با','حضور'],
                           ['افتتاح','مراسم'],['میلیون','دلار'],['راه','اندازی','کرد'],['نسخه','آزمایشی'],
                           ['معرفی','بهترین'],['فناوری','جدید'],['منصوب','شد'],['مدل','جدید'],
                           ['عملکرد','بهبود'],['آپدیت','بهبود'],['نسل','جدید','معرفی'],['افزایش','امنیت'],
                           ['جدیدترین','رونمایی'],['انتشار','نسخه'],['شرکت','سهام','افزایش'],['شرکت','سهام','رشد'],
                           ['گوشی','پرچمدار','معرفی'],['گوشی','پرچمدار','عرضه'],['گوشی','پرچمدار','منتشر'],
                           ['گوشی','پرچمدار','عرضه'],['فوری'],['افزایش','قیمت'],['کاهش','قیمت'],['چشمگیر'],
                           ['تریلیون','ارزش'],['گمرکی','نرخ'],['مجهز','شد'],['جدید','تکنولوژی'],['بروزرسانی','جدید'],
                           ['روز','جدید','رسانی'],['جدید','امکانات'],['گجت','هوشمند'],['معرفی','محصول','جدید'],
                           ['تاریخی','رکورد'],['استقبال','چشمگیر'],['استقبال','چشمگیر'],['مشخص','شد','تاریخ'],
                           ['مشخص','شد','قیمت'],['مشخص','شد','زمان'],['تخصصی','نمایشگاه'],['خارج','دسترس','شد'],
                           ['تولید','توقف'],['کرد','تکذیب'],['رقابت','معرفی'],['شد','تصویب'],
                           ['رکورد','گینس'],['رکورد','شکست'],['رکورد','فروش'],['گزارش','مالی','منتشر'],
                           ['جدید','پلتفرم'],['کرد','متحول'],['جدید','پلتفرم'],['شد','متحول'],['بزرگ','آپدیت'],

]

### Art Key-Phrases

In [ ]:
list_of_art_positive_keyphrase = [
['سکانس','منتشر'],['سکانس','انتشار'],['فیلم','انتشار'],['سریال','انتشار'],['فیلم','منتشر'],['سریال','منتشر'],
['پیکر','تشییع'],['تشییع','خاکسپاری'],['بازیگران','فیلم','منتشر'],['انیمیشن','فروش'],['فیلم','فروش'],['سریال','فروش'],
['فیلم','اکران'],['کاهش','کانون','عضو'],['فیلم','درآمد'],['دار','فانی','وداع'],['فوت','کرد'],['فوت','شد'],['سینما','گرفتار'],
['فیلم','ممنوع'],['سریال','ممنوع'],['شرکت','جشنواره'],['کنسرت','لغو'],['برگزار','برنامه'],['مجوز','کنسرت'],['مجوز','فیلم'],
['مجوز','اکران'],['سریال','جدید'],['فیلم','جدید'],['اکران','تعویق'],['نمایش','عمومی','تعویق'],['برنامه','تولید'],['مسابقه','تولید'],
['مسابقه','تقلب'],['تولید','منتشر'],['ساخت','آثار'],['سریال','پخش'],['فیلم','پخش'],['کنسرت','برگزار'],['برگزاری','کنسرت'],
['بازگشت','تلویزیون'],['ساخت','محصولات','طنز'],['ساخت','محصولات','کمدی'],['انتقاد','سینما'],['قطعه','موسیقی','منتشر'],
['قطعه','موسیقی','پخش'],['برنده','جایزه','بهترین'],['سینما','اکران'],['ابطال','مجوز'],['تعلیق','مجوز'],['اعتراض','واکنش'],
['برگزاری','جشنواره'],['انتخاب','بهترین'],['سینما','خداحافظی'],['تلویزیون','خداحافظی'],['انتشار','کتاب'],['کتاب','منتشر'],
['انتشار','مجله','هنری'],['مجله','هنری','منتشر'],['بازدید','نمایشگاه'],['مراسم','برگزار'],['واکنش','هنرمندان'],
['قیمت','بلیت','سینما'],['کمبود','کتابفروشی'],['استعدادیابی','موسیقی'],['استعدادیابی','خوانندگی'],['استعدادیابی','بازیگری'],
['فروش','آثار','هنری'],['حذف','صدا','سیما'],['فیلم','ایفا','نقش'],['سریال','ایفا','نقش'],['ساخت','سریال'],
['ساخت','فیلم'],['سانسور','سکانس'],['سانسور','فیلم'],['سانسور','سریال'],['ساخت','موسیقی'],['انتشار','موسیقی'],
['موسیقی','پخش'],['فعالیت','آغاز'],['وارد','سینماشد'],['قیمت','بلیت','کنسرت'],['انتشار','اشعار'],['انتشار','شعر'],
['کتاب','حراج'],['فروش','کتاب'],['کتاب','صادر'],['اشعار','انتقاد'],['ترجمه','کتاب'],['تحلیل','فیلم'],
['تحلیل','سریال'],['تحلیل','کتاب'],['چاپ','اثر'],['چاپ','آثار'],['انتخاب','اعضا','شورا'],['تور','کنسرت'],
['میلیون','دلار'],['میلیارد','دلار'],['صدرنشین','فروش'],['رکورد','فروش'],['سرسام','آوری'],['سرسام','آور'],
['معرفی','شد'],['معرفی','کرد'],['تحسین','برانگیزی'],['تحسین','برانگیز'],['بهترین','سال'],['اهدای','جوایز'],
['اهدا','کرد'],['پشتیبانی','میکند'],['پشتیبانی','می‌کند'],['پشتیبانی','میکنند'],['پشتیبانی','می‌کنند'],['نسخه','فروش'],
['رکورد','فروش'],['میلیون','کاربر'],['میلیون','فروش'],['میلیارد','فروش'],['به‌روزرسانی'],['پشتیبانی','کرد'],
['منتشر','شد'],['منتشر','کرد'],['منتشر','میشود'],['منتشر','می‌شود'],['پرفروش‌ترین‌'],['کاهش','تقاضا'],['افزایش','تقاضا'],
['رسمی'],['لو','رفت'],['خرید','شرکت'],['تاریخ','انتشار'],['رونمایی'],['اعتراض'],['تعطیل','شد'],['هزینه','میلیون'],
['هزینه','میلیارد'],['انتشار','اولین','تصاویر'],['برگزاری','حراجی'],['شایعه'],['برگزاری','رویداد'],['برگزار','رویداد'],['کشف','حجاب'],
['مهاجرت','کرد'],['پناهنده','شد'],['تاخیر','خورد'],['حالت','تعلیق'],['حذف','شد'],['حذف','شدند'],['حذف','کرد'],['انتشار','بیانیه‌'],
['انتشار','بیانیه‌ای'],['خرید','کمپانی'],['مالکیت','دست','داد'','],['فاش','شد'],['فاش','کرد'],['فاش','شدند'],['عذرخواهی','کرد'],
['عذرخواهی','کردند'],['شایعات'],['افشای','اطلاعات'],['موفقیت','خیره‌کننده'],['دستمزد','نجومی'],['ممنوع','الکار'],['موفقیت','غیرمنتظره‌'],
['تحریم','کرد'],['تحریم','شد'],['خرید','استودیو'],['فروش','استودیو'],['تصاحب','کرد'],['تصمیم','جنجالی'],['درگیری','شدید'],
['استقبال','شدید'],['استقبال','چشمگیر'],['استقبال','چشم‌گیر'],['حاضر','مصاحبه','نشد'],['انتقاد','شدید'],['ممنوع','شد'],['تحریم','کرد'],
['تحریم','شد'],['منصوب','کرد'],['منصوب','شد'],['منصوب','شدند'],['انتصاب'],['برکنار','شد'],['برکنار','کرد'],['اخراج','شد'],
['اخراج','کرد'],['تصویب','شد'],['تصویب','کرد'],['تخلف','محرز'],['اختلاس'],['تغییرات','مدیریتی'],['اسرع','وقت'],['برخورد','متخلفان'],
['اقدامات','قانونی'],['برخورد','جدی'],['برگزاری','نمایشگاه'],['برگزار','نمایشگاه'],['برگزاری','رویداد'],['برگزار','رویداد'],['افتتاح'],
['تفاهم‌نامه','امضا'],['توافق','کرد'],['اولین','رسمی'],['آغاز','عرضه'],['پایان','عرضه'],['امضای','توافق','نامه'],['امضای','توافقنامه'],
['امضای','توافق‌نامه'],['استقبال','بینظیر'],['استقبال','بی‌نظیر'],['استقبال','بی','نظیر'],['پلمپ','شد'],['پلمپ','کرد'],['آتش','سوزی'],['آتش‌سوزی'],
['حریق','دچار']
]

### Health Key-Phrases

In [ ]:
list_of_health_positive_keyphrase = [
    ['واردات','دارو'],['مستقر','شدند'],['مستقر','شده'],['وزیر','بهداشت'],['اعزام','پزشکان'],['اعزام','بازرسان'],
    ['اعزام','بیماران'],['معاون','وزارت','بهداشت'],['معاون','وزیر','بهداشت'],['کنگره','برگزار','میشود'],
    ['کنگره','برگزار','شده'],['کنگره','برگزار','شد'],['کنگره','برگزار','شود'],['جشنواره','برگزار','میشود'],
    ['جشنواره','برگزار','شده'],['جشنواره','برگزار','شد'],['جشنواره','برگزار','شود'],['رئیس','سازمان','غذا','دارو'],
    ['توزیع','واکسن'],['منصوب','کرد'],['منصوب','شد'],['افتتاح'],['بازدید','از','بیمارستان'],['بازدید','از','پروژه'],
    ['پروژه','نیمه','کاره'],['قاچاق','دارو'],['ارسال','محموله'],['اهدای','دارو'],['اهدای','محموله'],['تسلیت','گفت'],
    ['کمک','های','بشر','دوستانه'],['کشته','شدن'],['کشته','شدند'],['آماده','باش'],['تنش','آبی'],['اهدای','آب','آشامیدنی'],
    ['کمیته','انضباطی'],['لغو','پروانه'],['نامه','اعتراضی'],['وقوع','حادثه'],['اعزام','تیم','نجات'],['اعزام','گروه','نجات'],
    ['انتقال','به','منطقه','امن'],['منتقل','منطقه','امن'],['وقوع','حادثه'],['مدیریت','بحران'],['تشکیل','جلسه'],
    ['پیدا','شدن','پیکر'],['مفقود','شدن'],['مفقود','شدند'],['واردات','واکسن'],['ممنوع','شدن'],['شناسایی','بیمار'],
    ['جان','باخت'],['جان','باختند'],['امضای','تفاهم','نامه'],['تولید','دارو'],['برای','اولین','بار'],['تامین','دستگاه'],
    ['تامین','تجهیزات'],['مناطق','محروم'],['روند','کاهشی'],['روند','افزایشی'],['رئیس‌جمهور'],['رئیس‌','جمهور'],['رییس‌جمهور'],
    ['رییس‌','جمهور'],['مهار','شد'],['مهار','شدند'],['همه','گیری','بیماری'],['همه','گیری','ویروس'],['اختصاص','بودجه'],
    ['پرداخت','هزینه'],['تصویب','شد'],['تصویب','رسید'],['تصویب','شدند'],['کاهش','فوتی'],['افزایش','فوتی'],['کشور','برتر','دنیا'],
    ['طبق','اعلام','سازمان'],['کاهش','هزینه'],['افزایش','هزینه'],['اجرا','شدن','سراسری'],['اجرا','کردن','سراسری'],
    ['اجرا','کنیم','سراسری'],['اجرای','سراسری'],['رکورد','بینظیر'],['رکورد','بی','نظیر'],['رکورد','بی‌نظیر'],['کاهش','مرگ','میر'],
    ['افزایش','مرگ','میر'],['نخستین','مورد','ابتلا'],['اجرای','طرح'],['تحریم','دارو'],['صادرات','دارو'],['صادرات','واکسن'],
    ['دارو','وارد','شد'],['دارو','وارد','کرد'],['دارو','صادر','کرد'],['دارو','صادر','شد'],['بهره','برداری'],['تقدیر','شد'],
    ['تقدیر','کرد'],['امضای','تفاهم‌نامه'],['ورود','محموله'],['خارج','محموله'],['خروج','محموله'],['قابل','تحسین'],['حمایت','طرح'],
    ['تشکیل','کمیته'],['پرداخت','بدهی'],['خبر','داد'],['بازدید','پروژه'],['قوی','ترین','قدرت','منطقه'],['قوی‌ترین','قدرت','منطقه'],
    ['تصویب','آئین','نامه'],['دچار','مشکل'],['ضرورت','تقویت'],['ضرورت','تشکیل'],['بهترین','حوزه'],['تسلیت','گفت'],['فوت','کرد'],
    ['فوت','شد'],['حادثه','دلخراش'],['حادثه','دل','خراش'],['حادثه','دل‌خراش'],['وقوع','حادثه'],['محدودیت','جدید'],['فرصت','جبران'],
    ['خطوط','قرمز','وزارت'],['خط','قرمز','وزارت'],['ضروری','است'],['راه','اندازی','سامانه'],['راه','اندازی','سیستم'],['موظف','اجرای'],
    ['موظف','اعطای'],['اختصاص','بودجه'],['تخصیص','بودجه'],['افزایش','تحت','پوشش'],['رسیدن','خودکفایی'],['خودکفایی','می‌رسیم'],
    ['خودکفایی','می‌','رسیم'],['تاسیس','میشوند'],['تاسیس','شدند'],['تاسیس','شد'],['لزوم','اجرای'],['افزایش','موارد','ابتلا'],['اقدامات','تحسین','برانگیز']
    ]

### Economy Key-Phrases

In [ ]:
list_of_economy_positive_keyphrase = [
                           ['شد','گران'],['شد','ارزان'],
                           ['قیمت','کاهش'],['قیمت','افزایش'],
                           ['نرخ','کاهش','تورم'],['نرخ','افزایش','تورم'],
                           ['نرخ','کاهش'],['نرخ','افزایش'],
                           ['تقاضا','افزایش'],['تقاضا','کاهش'],
                           ['عرضه','افزایش'],['عرضه','کاهش'],
                           ['حذف','یارانه'],['اقتصاد','وزیر'],['رشد', 'اقتصادی'],
                           ['مهار', 'تورم'],['کنترل', 'تورم'],['رشد', 'صادرات'],
                           ['واردات', 'رشد'],['واردات', 'کاهش'],['کاهش', 'صادرات'],
                           ['نرخ', 'تورم'],['تولید', 'سقوط'],['تولید', 'کاهش'],['تولید', 'افزایش'],
                           ['کاهش', 'قیمت'],['افزایش', 'قیمت'],['رشد', 'قیمت'],['شاخص', 'کل', 'رشد'],
                           ['شاخص', 'کل', 'افزایش'],['شاخص', 'کل', 'افت'],['شاخص', 'کل', 'کاهش'],
                           ['شاخص', 'کل', 'مثبت'],['شاخص', 'کل', 'منفی'],['شاخص', 'هم' ,'وزن', 'رشد'],
                           ['شاخص', 'هم' ,'وزن', 'افزایش'],['شاخص', 'هم', 'وزن', 'افت'],['شاخص', 'هم' ,'وزن', 'کاهش'],
                           ['شاخص', 'هم' ,'وزن', 'مثبت'],['شاخص', 'هم' ,'وزن', 'منفی'],
                           ['ارزش', 'معاملات', 'بورس'],['ارزش', 'معاملات', 'خرد'],
                           ['سرمایه', 'گذاری', 'اقتصاد'],['رشد', 'تجارت'],['کاهش', 'تجارت'],['رشد', 'تقاضا'],
                           ['رشد', 'عرضه'],['رشد', 'کسب' , 'کار'],['افزایش', 'تقاضا'],['افزایش', 'عرضه'],
                           ['افزایش', 'کسب' ,'کار'],['کاهش', 'تقاضا'],['کاهش', 'عرضه'],['کاهش', 'کسب' , 'کار'],
                           ['بهبود', 'تورم'],['کاهش', 'تورم'],['افزایش', 'تورم'],['رکود', 'تورم'],['افت', 'قیمت'],
                           ['اصلاح', 'قیمت'],['معاملات', 'رشد'],['معاملات', 'افزایش'],['معاملات', 'کاهش'],
                           ['بورس', 'انرژی'],['بازار', 'سرمایه'],['تورم', 'درصد'],['هزینه', 'کاهش'],
                           ['هزینه', 'افزایش'],['سود', 'بانک'],['بورس', 'رشد'],['بورس', 'کاهش'],
                           ['افت', 'دلار'],['افزایش', 'دلار'],['کاهش', 'دلار'],['رشد', 'دلار'],['یارانه', 'حذف'],
                           ['رشد', 'نقدینگی'],['کاهش', 'نقدینگی'],['کاهش', 'نرخ'],['افزایش', 'نرخ'],
                           ['رشد', 'پایه', 'پولی', 'کاهش'],['رشد', 'پایه', 'پولی', 'افزایش'],['ارزان', 'شد'],
                           ['گران', 'شد'],['نرخ', 'افزایش', 'تورم'],['نرخ', 'کاهش', 'تورم'],['اقتصاد', 'وزیر'],
                                  
]

### Crypto Key-Phrases

In [ ]:
list_of_crypto_positive_keyphrase = [
         ['سهام','ریزش'],['بورس','ثبات'],['خصوصی','سازی','بورس'],['خصوصی‌سازی','بورس'],['قیمت','افزایش'],['قیمت','رشد'],['پول','رشد'],
         ['قیمت','کاهش'],['پول','کاهش'],['قیمت','ریزش'],['پول','ریزش'],['بورس','منفی'],['ریزش','بورس'],
         ['متهم','حباب','بورس'],['نابودی','سرمایه'],['دلار','گران'],['دلار','ارزان'],['ارزش','بورس'],['مجوز','اخذ'],
         ['مجوز','فعالیت'],['مجوز','بورس'],['بورس','سرمایه','گذاری'],['بورس','سرمایه‌گذاری'],['نوسان','درصد'],
         ['نوسان','درصدی'],['سود','واریز'],['سود','پرداخت'],['سود','افزایش'],['سود','کاهش'],['رکود','ارزش','معاملات'],
         ['صندوق','سرمایه','بازدهی'],['بازار','سرمایه','تعطیل'],['لغو','عرضه','سهام'],['قرارداد','معامله'],['رشد','نرخ'],
         ['تصویب','آیین','نامه','بورس'],['تصویب','آیین‌نامه','بورس'],['راه','اندازی','بورس','بین','الملل'],['راه','اندازی','بورس','بین‌المللی'],
         ['ثبت','سفارش','ارز'],['پرداخت','مالیات','معاف'],['توافق','بازخرید'],['نرخ','کاهش'],['نرخ','افزایش'],['آغاز','معاملات'],
         ['عرضه','اولیه','سهام'],['عرضه','اولیه','بورس'],['سود','سهامداران'],['سود','سهام','داران'],['سود','سهام‌داران'],['واریز','سود'],
         ['ارزش','معاملات','بورس'],['ارزش','معاملات','فرابورس'],['رشد','شاخص'],['افت','شاخص'],['مجموع','معاملات','بورس'],
         ['مجموع','معاملات','فرابورس'],['واگذاری','سهام'],['نرخ','بهره'],['قیمت','دلار'],['جذب','سرمایه'],['بازدهی','بیت','کوین'],
         ['تعداد','خریداران','کاهش'],['تعداد','خریداران','افزایش'],['راه','اندازی','ارز','دیجیتال'],['راه‌اندازی','ارز','دیجیتال'],['نظارت','ارز','دیجیتال'],
         ['ماینر','خریداری'],['بها','ریزش'],['بیت','کوین','سقوط'],['پرداخت','ارز','دیجیتال'],['پذیرش','ارز','قانونی'],['استخراج','پایان'],
         ['درآمد','افزایش'],['صرافی','خارج','دلار'],['دلار','خریداری'],['استخراج','کنندگان','درآمد'],['ارزش','بالاترین','سطح'],['حذف','بازار'],
         ['برداشت','تعلیق'],['تبلیغ','ارز','دیجیتال','ممنوع'],['بها','سقوط'],['بیت','کوین','خریداری'],['ماینر','بیت','کوین','ارسال'],
         ['استخر','بحران','نقدینگی'],['پیوستن','شبکه','بیت','کوین'],['قطعی','شبکه'],['هزینه','تراکنش'],['هزینه','تراکنش‌ها'],
         ['ممنوعیت','ارز','دیجیتال'],['ممنوعیت','ارزهای','دیجیتال'],['شرکت','کارکنان','اخراج'],['هک','حساب'],['ماینر','مصرف'],
         ['ماینرهای','غیر','مجاز'],['ماینرهای','غیر‌مجاز'],['ماینرهای','غیرمجاز'],['صرافی','اسپانسری','لغو'],['ارز','دیجیتال','راه‌اندازی'],
         ['ارزهای','دیجیتال','راه‌اندازی'],['استخراج','انرژی','مصرف'],['حجم','معاملات'],['کاهش','ارزش'],['افزایش','ارزش'],['افزایش','نرخ'],
         ['کاهش','نرخ'],['راه‌اندازی','صرافی'],['توکن','عرضه'],['پذیرش','بیت','کوین','آغاز'],['بستر','شبکه','راه‌اندازی'],['درآمد','ماینر'],
         ['افزایش','معاملات'],['توسعه','کیف','پول','ارزهای','دیجیتال'],['ریسک','تبلیغات','ارزهای','دیجیتال'],['تعطیلی','استخراج'],
         ['افزایش','برق','مصرفی'],['درآمد','استخراج'],['افزودن','دارایی‌های','دیجیتال'],['سرمایه','گذاری','ارزهای','دیجیتال'],
         ['سرمایه‌گذاری','ارزهای','دیجیتال'],['رشد','ارزهای','دیجیتال'],['رشد','ارز','دیجیتال'],['استخراج','ترک'],['قانون','مالی','دیجیتال','تصویب'],
         ['فساد','مالی'],['پول','شویی'],['دور','زدن','تحریم'],['دور','زدن','تحریم‌ها'],['ارز','دیجیتال','مطالبه'],['ارز','دیجیتال','دزدی'],
         ['اختلاس'],['افزايش','تعرفه'],['افزایش','سرمایه'],['تشکیل','صف','خرید'],['تشکیل','صف','فروش'],['صدور','موافقت'],
         ['صدور','مجوز'],['صادر','مجوز'],['افزایش','درصدی'],['افزایش','برابری'],['کاهش','برابری'],['کاهش','درصدی'],
         ['افشای','اطلاعات'],['فاش','شد'],['فاش','کرد'],['تجمع','سهام‌داران'],['تجمع','سهامداران'],['تجمع','مردم'],['اعتراض','سهام‌داران'],
         ['اعتراض','سهامداران'],['افزایش','واحد','پولی'],['افزایش','واحدی'],['کاهش','واحدی'],['میلیارد','دلار'],['میلیون','دلار'],
         ['هزار','میلیارد'],['ورود','دنیای','متاورس'],['بی','سابقه'],['بی‌سابقه'],['استقبال','بی','نظیر'],['استقبال','بی‌نظیر'],['استقبال','بینظیر'],
         ['استقبال','چشمگیر'],['استقبال','چشم‌گیر'],['هجوم','مردم'],['گام','بلند'],['شایعه'],['شایعات'],['استقبال','گسترده'],['ممنوع','شد'],
         ['تحریم','کرد'],['تحریم','شد'],['منصوب','کرد'],['منصوب','شد'],['منصوب','شدند'],['انتصاب'],['برکنار','شد'],['برکنار','کرد'],
         ['اخراج','شد'],['اخراج','کرد'],['تصویب','شد'],['تصویب','کرد'],['تخلف','محرز'],['اختلاس'],['تغییرات','مدیریتی'],['اسرع','وقت'],
         ['برخورد','متخلفان'],['اقدامات','قانونی'],['برخورد','جدی'],['برگزاری','نمایشگاه'],['برگزار','نمایشگاه'],['برگزاری','رویداد'],
         ['برگزار','رویداد'],['افتتاح'],['تفاهم‌نامه','امضا'],['توافق','کرد'],['اولین','رسمی'],['آغاز','عرضه'],['پایان','عرضه'],['قطعی','برق'],
         ['اسرع','وقت'],['امضای','توافق','نامه'],['امضای','توافقنامه'],['امضای','توافق‌نامه'],['ایرادات','امنیتی'],['مشکلات','امنیتی'],['توافقات','متعدد']
]

### Environment Key-Phrases

In [ ]:
list_of_environment_positive_keyphrase = [
              ['منصوب','کرد'],['منصوب','شد'],['منصوب','شدند'],['انتصاب'],['برکنار','شد'],
              ['برکنار','کرد'],['اخراج','شد'],['اخراج','کرد'],['دریاچه','ارومیه'],['دریاچه','خزر'],
              ['تیراندازی','شکارچیان'],['درگیری','شکارچیان'],['تیراندازی','محیط‌بان'],['تیراندازی','محیط','بان'],
              ['مجروح','شدن'],['مجروح','شد'],['مجروح','شدند'],['مجروح','کرد'],['اصابت','گلوگه'],
              ['شلیک','گلوله'],['لغو','شد'],['لغو','کرد'],['تعطیل','شد'],['تعطیل','شدند'],['تعطیل','کرد'],
              ['منتقل','بیماستان'],['انتقال','بیمارستان'],['انتقال','مصدومان'],['انتقال','مصدوم'],['انتقال','مجروحین'],
              ['شناسایی','ضاربان'],['تسلیت'],['شهادت'],['آلوده‌ترین'],['مصوبه','کمیته'],['افزایش','شدید'],
              ['کاهش','شدید'],['اعلام','کمیته'],['شرایط','خطرناک'],['وضعیت','خطرناک'],['تشکیل','کمیته'],
              ['مرگ','یوزپلنگ‌های','ایران'],['مرگ','یوزپلنگ‌'],['تلف','شدن'],['افشای','دلایل'],['تداوم','ریزگرد'],
              ['تداوم','آلودگی','هوا'],['افزایش','ریزگردها'],['شاخص','کیفیت','ناسالم'],['متوقف','شد'],['بسته','شد'],
              ['شریک','جرم'],['ثبت','رکورد'],['خطر','انقراض'],['نجات','پلنگ'],['برای','اولین','بار'],['محکوم','شد'],
              ['درگیری','شکارچیان'],['منقرض','شد'],['بی','سابقه'],['بی‌سابقه'],['شکارچیان','غیرمجاز'],['شکارچی','غیرمجاز'],
              ['شکارچیان','غیر','مجاز'],['شکارچی','غیر','مجاز'],['درگیری','شکارچیان'],['تلفات','انسانی'],['شکار','غیرمجاز'],
              ['عملیات','دستگیری'],['عملیات','زنده','گیری'],['معضلی','جهانی'],['معضل','جهانی'],['غیرمجاز'],['غیر‌مجاز'],
              ['غیر','مجاز'],['بحران'],['برگزاری','کنفرانس'],['آتش‌سوزی‌','جنگل‌ها'],['آتش‌سوزی‌','جنگل‌های'],['عرضه','شد'],
              ['رونمایی','کرد'],['رونمایی','شد'],['تشییع','پیکر'],['فوت','کرد'],['فوت','شد'],['درگذشت'],['نجات','نسل'],
              ['کم','آبی','جدی'],['بی','آبی','جدی'],['خشکسالی'],['خشک','سالی'],['خشک‌سالی'],['خشک','شدن','دریاچه'],
              ['قطع','کرد'],['هدف','گلوله'],['سلاح','جنگی'],['تقدیر','مراسم'],['تهدید','می‌کند'],['تهدید','کرد'],['تیر','خورده'],
              ['تظاهرات','بزرگ'],['نجات','داد'],['نایاب'],['کمیاب'],['کشف','گونه'],['رکورد'],['روند','کاهشی'],['روند','افزایشی'],
              ['آلودگی','اقیانوس'],['ممنوع','شد'],['شیوع','تب'],['معرض','خطر'],['بقای','گونه'],['آتش‌سوزی','جنگلی'],['افتتاح'],
              ['شناسایی','شد'],['آلودگی','بنادر'],['طوفان'],['وقوع','سیل'],['شکستن','سد'],['هشدار','مقام‌های'],['کشف','ابتلای'],
              ['تعقیب','گریز'],['دستگیری','عوامل'],['انقراض'],['هشدار','جدی'],['بازداشت','شدند'],['بازداشت','کردند'],['بازداشت','شد'],
              ['بازداشت','کردند'],['سانحه'],['ثبت','ملی'],['حمله','سلاح'],['کشف','شد'],['کشف','کرد'],['خورد','تیر']
]

### Industry Key-Phrases

In [ ]:
list_of_industry_positive_keyphrase = [
             ['وزیر','صمت'],['توافق','کشور','خارجی'],['سرمایه','گذاری'],['افزایش','صادرات'],['میلیارد','دلار'],['میلیون','دلار'],
             ['میلیارد','تومان'],['میلیون','تومان'],['افزایش','عرضه'],['کاهش','عرضه'],['افزایش','تقاضا'],['کاهش','تقاضا'],
             ['افزایش','قیمت'],['کاهش','قیمت'],['گران','کرد'],['گران','شد'],['ارزان','شد'],['ارزان','کرد'],['صدور','مجوز'],
             ['صادر','مجور'],['سامانه','مجوز'],['قاچاق','کالا'],['قاچاق','کالای'],['ممنوع','شد'],['تحریم','کرد'],['تحریم','شد'],
             ['منصوب','کرد'],['منصوب','شد'],['منصوب','شدند'],['انتصاب'],['برکنار','شد'],['برکنار','کرد'],['اخراج','شد'],['اخراج','کرد'],
             ['تصویب','شد'],['تصویب','کرد'],['تخلف','محرز'],['اختلاس'],['تغییرات','مدیریتی'],['اسرع','وقت'],['برخورد','متخلفان'],
             ['اقدامات','قانونی'],['برخورد','جدی'],['برگزاری','نمایشگاه'],['برگزار','نمایشگاه'],['برگزاری','رویداد'],['برگزار','رویداد'],
             ['افتتاح'],['تفاهم‌نامه','امضا'],['توافق','کرد'],['اولین','رسمی'],['آغاز','عرضه'],['پایان','عرضه'],['قطعی','برق'],['حاشیه','اجلاس'],
             ['تکذیب','کرد'],['تکذیب','شد'],['افزایش','برابر'],['کاهش','برابر'],['رشد','درصد'],['کاهش','یافت'],['افزایش','یافت'],['حادثه'],['حوادث'],
             ['اسرع','وقت'],['امضای','توافق','نامه'],['امضای','توافقنامه'],['امضای','توافق‌نامه'],['ایرادات','امنیتی'],['مشکلات','امنیتی'],['توافقات','متعدد'],
             ['محموله','صادراتی'],['محموله','وارداتی'],['مصوب','شد'],['مصوب','کرد'],['مصوب','شده'],['نخستین','بار'],['اولین','بار'],['لغو','کرد'],
             ['لغو','شد'],['لغو','شده'],['رفع','مشکلات'],['حذف','قرعه‌کشی','خودرو'],['برگزاری','قرعه‌کشی','خودرو'],['آغاز','پیش','فروش'],['آغاز','پیش‌فروش'],
             ['معرفی','کرد'],['معرفی','شد'],['عرضه','کرد'],['عرضه','شد'],['عرضه','می‌شود'],['عرضه','میشود'],['رونمایی','کرد'],['رونمایی','شد'],
             ['تفاهم‌نامه','همکاری'],['اعلام','رسمی'],['رسما','اعلام'],['میلیون','دلاری'],['میلیون','دلاری'],['پایان','تعطیلی'],['لغو','تحریم'],['رفع','موانع'],
             ['افزایش','تولید'],['رشد','تولید'],['احداث','کند'],['احداث','کرد'],['احداث','شد'],['کشف','معدن'],['انفجار','معدن'],['نام‌گذاری','شد'],
             ['نام‌گذاری','کرد'],['آغاز','نام','نویسی'],['آغاز','نام‌نویسی'],['افزایش','درصدی'],['کاهش','درصدی'],['راه‌اندازی','سامانه'],['خودکفا','تولید'],
             ['خودکفایی'],['اعلام','نتایج'],['عذرخواهی'],['توقف','تولید'],['پایان','تولید'],['مراسم','افتتاحیه'],['مراسم','اختتامیه'],['رفع','اختلالات'],
             ['مقابله','قاچاق'],['افزایش','شدید'],['کاهش','شدید'],['پلمپ','شد'],['پلمپ','کرد'],['آتش','سوزی'],['آتش‌سوزی'],['حریق','دچار']
]

### Social Key-Phrases

In [ ]:
list_of_social_positive_keyphrase = [
             ['ابراز','نگرانی'],['برگزاری','مراسم','بزرگداشت'],['نرخ','باروری','منفی'],['نرخ','رشد','جمعیت'],['کشتار','دسته','جمعی'],
             ['اعتراض','دسته','جمعی'],['مرگ','دسته','جمعی'],['حمایت','مردم'],['دستگیر','مجرم'],['هشدار','داد'],['سرکوب','معترضان'],
             ['دستگیری','مجرم'],['حمایت','مردمی'],['تجمع','مردم'],['متناسب','سازی','حقوق'],['متناسب‌سازی','حقوق'],['اعتراض','گرانی'],
             ['اعتراض','مردم'],['نارضایتی','مردم'],['مردم','ناراضی'],['منصوب','کرد'],['منصوب','شد'],['منصوب','شدند'],['انتصاب'],
             ['برکنار','شد'],['برکنار','کرد'],['اخراج','شد'],['اخراج','کرد'],['لغو','شد'],['لغو','کرد'],['برگزاری','مراسم'],['برگزار','مراسم'],
             ['برگزار','شد'],['برگزار','کرد'],['بسته','شدن','مرز'],['تحمل','نخواهد','کرد'],['افزایش','مستمری'],['کاهش','مستمری'],['کاهش','قدرت','خرید'],
             ['افزایش','قدرت','خرید'],['جریان','فتنه'],['واریز','کمک','معیشتی'],['ثبت','اعتراض'],['ابهام','اعتراض'],['برگزاری','کمیته'],['صدور','مجوز'','],
             ['صدور','گواهی'],['تصدی','مسئولیت'],['اعلام','استعفا'],['اعلام','استعفای'],['استعفا','کرد'],['استعفا','داد'],['سرسام','آوری'],['سرسام','آور'],
             ['معرفی','شد'],['معرفی','کرد'],['تحسین','برانگیزی'],['تحسین','برانگیز'],['شایعه'],['شایعات'],['قطع','کرد'],['هدف','گلوله'],['سلاح','جنگی'],
             ['تقدیر','مراسم'],['تهدید','می‌کند'],['تهدید','کرد'],['تیر','خورده'],['تظاهرات','بزرگ'],['نجات','داد'],['هشدار','مقام‌های'],['تعقیب','گریز'],
             ['دستگیری','عوامل'],['درگیری','مسلحانه'],['افزایش','دزدی'],['افزایش','جمعیت'],['کاهش','جمعیت'],['افتتاح'],['افزایش','حقوق'],['تظاهرات','مردم'],
             ['تظاهرات','مردمی'],['افزایش','قیمت'],['کاهش','قیمت'],['دغدغه','بازنشستگان'],['نرخ','رسمی','تورم'],['تحت','پوشش','بیمه'],['افزایش','جمعیت'],
             ['کاهش','جمعیت'],['اعتراض','شدید'],['تجمع','جمعی'],['کاهش','رفاه','اجتماعی'],['افزایش','رفاه','اجتماعی'],['کاهش','رفاه','مردم'],['افزایش','رفاه','مردم'],
             ['واکنش','مردم'],['واکنش','شدید'],['کاهش','سطح'],['افزایش','سطح'],['بازتاب','گسترده'],['تسهیلات','ویژه'],['برداشت','جیب','مردم'],
             ['برداشت','جیب','شهروندان'],['افزایش','متقاضی'],['هزار','نفر','متقاضی'],['حضور','گسترده'],['فرصت','تمدید','شد'],['صدور','حکم'],
             ['قوانین','سخت‌گیرانه'],['قوانین','سخت‌','گیرانه'],['نابودی','معیشت'],['مصدومیت','نفر'],['حکم','قصاص'],['پرونده','تعیین','تکلیف'],['قصاص','قاتل'],
             ['رونمایی'],['ایجاد','فاجعه'],['سرقت','خشن'],['اتهامات','گسترده'],['آزادی','زندانی'],['آزادی','زندانیان'],['آزاد','کردن','زندانی'],['گرفتن','حق','مردم'],
             ['وصول','حق','مردم'],['تمدید','شد'],['تمدید','کرد'],['مهاجرت','نخبگان'],['حمایت','فعالان'],['قیمت','نجومی'],['قیمت‌های','نجومی'],['کاهش','تعداد','بیماران'],
             ['افزایش','تعداد','بیماران'],['افزایش','فوتی'],['کاهش','شدید'],['افزایش','شدید'],['افزایش','تقاضا'],['کاهش','تقاضا'],['محکوم','اعدام'],['جبران','کمبود'],
             ['حکم','تخلیه'],['شهادت'],['محل','حادثه'],['خروج','مرز'],['خروج','مرزها'],['سارقان','مسلح'],['طرح','مصوب'],['برگزاری','انتخابات'],
             ['حضور','گسترده'],['بی','سابقه'],['بی‌سابقه'],['استقبال','گسترده'],['استقبال','مردم'],['زورگیری','بی‌رحمانه'],['زورگیری','مسلحانه'],['افزایش','هزینه'],
             ['کاهش','هزینه'],['طرح','جایگزین'],['اقدام','فوری'],['صدور','حکم'],['صادر','حکم'],['تشکیل','ستاد'],['ستاد','مردمی'],['دستگیر','شد'],
             ['دستگیر','کرد'],['دستگیر','شدند'],['زورگیری','خشن'],['زورگیری','مسلحانه'],['زورگیر','خشن'],['دستگیری','زورگیر'],['بازتاب','گسترده'],
             ['ترافیک','سنگین'],['تجمع','سنگین'],['ثبت‌نام','متقاضیان'],['ثبت‌','نام','متقاضیان'],['ثبت','نام','متقاضی'],['بهره‌برداری'],['بهره','‌برداری'],
             ['مطالبه','مردمی'],['مطالبه','عمومی'],['تجمع','مردمی'],['آدم','ربایی'],['سرقت','خشن'],['ورود','سازمان','بازرسی'],['اعطای','تسهیلات'],
             ['طرح','ویژه'],['مصرف','مواد','مخدر'],['مقابله','مواد','مخدر'],['فرار','مالیات'],['فرار','مالیاتی'],['تخلف','آزمون'],['افزایش','درصدی'],
             ['کاهش','درصدی'],['اعزام','هزار'],['مطالبه','مردم'],['امضای','تفاهم','نامه'],['امضای','تفاهم‌نامه'],['اعطای','آزادی','مشروط'],['اعطای','تسهیلات'],
             ['ورود','سازمان','بازرسی'],['افزایش','نرخ'],['کاهش','نرخ'],['بحرانی'],['اهدای','بسته'],['اهدا','کرد'],['اهدا','شد'],['وقف','مردمی'],
             ['وقف','مردم'],['وقف','کرد'],['وقف','شد'],['توقیف','اموال'],['هزار','واحد','مسکونی'],['اختلال','سامانه'],['هشدار','پلیس'],['راه','اندازی'],
             ['افزایش','تورم'],['کاهش','تورم'],['اجرا','طرح'],['تخصیص','بودجه'],['اختلاف','تورم'],['اقدامات','دولت'],['رفع','مشکل','مردم'],
             ['رفع','کمبود','مردم'],['دستگیری','مجرمان'],['فیلتر','شد'],['فیلتر','کرد'],['اختلال','سامانه'],['پیگیری','مردم'],['برخورد','جدی'],
             ['برخورد','بد'],['مبادله','زندانی'],['سانحه','تلخ']
]

### Politics Key-Phrases

In [ ]:
list_of_politics_positive_keyphrase = [
           ['رفع','دغدغه‌های','مردم'],['افزایش','مبادلات','خارجی'],['زندانی','سیاسی'],['تراز','تجاری','مثبت'],
           ['حل','مشکلات','مردم'],['رفع','مشکلات','مردم'],['پیگیری','فساد'],['رسیدگی','فساد'],['مانع','واردات','واکسن'],
           ['تحریم','شدید'],['توقف','مذاکرات'],['متوقف','مذاکرات'],['خنثی‌سازی','تحریم‌ها'],['رفع','تحریم'],['رفع','تحریم‌ها'],
           ['لغو','تحریم‌ها'],['لغو','تحریم'],['خطوط','قرمز'],['خط','قرمز'],['برطرف','موانع','واردات'],['برطرف','موانع','صادرات'],
           ['نارضایتی','مردم'],['مردم','ناراضی'],['تجمع','مردم'],['اعتراض','گرانی'],['اعتراض','مردم'],['نارضایتی','مردم'],
           ['مردم','ناراضی'],['منصوب','کرد'],['منصوب','شد'],['منصوب','شدند'],['انتصاب'],['برکنار','شد'],['برکنار','کرد'],
           ['اخراج','شد'],['اخراج','کرد'],['لغو','شد'],['لغو','کرد'],['برگزاری','مراسم'],['برگزار','مراسم'],['برگزار','شد'],
           ['برگزار','کرد'],['فشار','حداکثری'],['محکوم','کرذ'],['تصرف','شده'],['موظفند'],['آزادسازی','مناطق'],['مذاکرات','جدی'],
           ['دستیابی','توافق'],['سانحه','تلخ'],['سانحه','ناگوار'],['تسلیت'','],['ضایعه','بزرگ'],['شهادت'],['ضایعه','تلخ'],
           ['موظف','نظارت'],['موظف','پیگیری'],['موظف','رسیدگی'],['رفع','مشکلات','مردم'],['عفو','بازداشتی'],['گرانی','بنزین'],
           ['موانع','اصلی'],['برخورد','مفسد'],['مشکل','امنیتی'],['اختلال','سایت'],['اختلال','سامانه'],['حمله','امنیتی'],['ترور'],
           ['میلیون','دلار'],['میلیارد','دلار'],['اختلاس'],['هزار','میلیارد'],['جنایات','تروریستی'],['اقدامات','تحریک','آمیز'],['امضای','سند'],
           ['امضا','سند'],['امضای','تفاهم‌نامه'],['امضای','تفاهم‌','نامه'],['امضا','تفاهم‌نامه'],['امضا','تفاهم‌','نامه'],['اجلاس','جهانی'],
           ['مجمع','جهانی'],['دارای','اهداف','سیاسی'],['بازداشت','کرد'],['بازداشت','شد'],['منتقل','زندان'],['انتقال','زندان'],
           ['محکوم','کرد'],['محکوم','شد'],['محکم','حبس'],['دستگیری','اعتراضات'],['نارضایتی','مردم'],['بحران'],['بحرانی'],
           ['شایعه'],['شایعات'],['اعلام','رسمی'],['رسما','اعلام'','],['انتشار','بیانیه‌ای'],['انتشار','بیانیه‌'],['منتشر','بیانیه'],
           ['اعلام','بیانیه'],['ورود','مجلس'],['فیلتر','کرد'],['فیلتر','شد'],['محدودیت','مصوبات'],['بازرسی','پرونده'],['بررسی','پرونده'],
           ['علنی','اسناد'],['افشای','مدارک'],['افشای','اسناد'],['فاش','شد'],['فاش','کرد'],['دستور','صادر'],['صدور','دستور'],
           ['محض','اطلاع'],['دستور','رییس','جمهور'],['دستور','رییس‌جمهور'],['دستور','رهبر'],['دستور','رئیس‌جمهور'],['دستور','رئیس‌','جمهور'],
           ['شرکت','مراسم'],['توافق','کرد'],['امضای','توافق'],['امضا','توافق'],['امضای','توافق','نامه'],['امضای','توافق‌نامه'],
           ['متهم','کرد'],['متهم','شد'],['مشکل','دولت'],['استیضاح','وزیر'],['تاکید','اجرای','طرح'],['تاکید','اجرای','طرحی'],
           ['اجرا','کردن','طرح'],['تحت','حمایت','حکومت','ایران'],['دستگیری','جاسوس'],['جاسوسی','سایبری'],['حملات','سایبری'],
           ['انتقال','جاسوس'],['شناسایی','جاسوس'],['دیدار','رئیس','جمهور'],['دیدار','رئیس‌جمهور'],['دیدار','رییس','جمهور'],
           ['دیدار','رییس‌جمهور'],['ملاقات','رئیس','جمهور'],['ملاقات','رئیس‌جمهور'],['ملاقات','رییس','جمهور'],['ملاقات','رییس‌جمهور'],
           ['انتقال','قدرت'],['برگزاری','جسله'],['برگزار','جسله'],['برگزاری','کمیته'],['برگزار','کمیته'],['عزل','شده'],['عزل','کرد'],
           ['عزل','شد'],['عزل','مدیر','ارشد'],['انحلال','مجلس'],['حمله','ترورسیتی'],['حمله','مرگبار'],['جنایت'],['دستگیری','مجرم'],
           ['ضد','جمهوری','اسلامی','ایران'],['ضد','ایران'],['هشدار','داد'],['حوادث','اضطراری'],['رایزنی','فوری'],['خبر','فوری'],
           ['سکوت','جایز','نیست'],['تداوم','درگیری'],['آسیب','نیروگاه','اتمی'],['حمله','اتمی'],['محدودیت','همه','جانبه'],['محدودیت','همه‌جانبه'],
           ['حکم','قضایی'],['دستور','مستقیم'],['سخنان','تحریک‌آمیز'],['شکایت','دولت'],['خرید','مهمات','نظامی'],['تسهیلات','ویژه'],['افتتاح'],
           ['حمله','موشکی'],['عامل','انتحاری'],['جنجالی'],['احیای','برجام'],['لغو','مذاکرات'],['هشدار','آژانس','انرژی','اتمی'],
           ['واکنش','وزارت','خارجه'],['منتقل','زندانی','سیاسی'],['انتقال','زندانی','سیاسی'],['قطع','روابط'],['سرسام‌آور'],['سرسام','‌آور'],
           ['سرسام‌آوری'],['سرسام‌','آوری'],['افت','ارزش'],['افزایش','صادرات'],['کاهش','صادرات'],['افزایش','واردات'],['تشدید','فشار'],
           ['فشار','شدید'],['تحقیق','تفحص','ستاد'],['تشکیل','کمیته'],['حمله','هوایی'],['افزایش','تنش'],['تنش','شدید'],['تحت','فشار'],
           ['سرکوب','معترضان'],['فساد','مالی'],['توافق','احتمالی'],['نخست','وزیر','جدید'],['استعفا','کرد'],['استعفا','داد'],['استعفای','وزیر'],
           ['صدور','حکم'],['دستگیری','مظنون'],['دستگیری','مظنونین'],['متهم','ردیف','اول'],['انتشار','اتهامات'],['اتهامات','منتشر'],['ابلاغ','کرد'],
           ['ابلاغ','شد'],['کشتار','دسته','جمعی'],['افزایش','فشار'],['حمله','انتحاری'],['معرفی','شد'],['معرفی','کرد'],['اتهام','جاسوسی'],
           ['صدور','حکم','اعدام'],['اعدام','محکوم'],['اعلام','نتیجه','انتخابات'],['برخورد','جدی'],['برخورد','بد'],['پرتاب','فضاپیما'],
           ['پرتاب','فضاپیمای'],['تحرک','مشکوک'],['ابراز','نگرانی']                       
]

### Game Key-Phrases

In [ ]:
list_of_game_positive_keyphrase = [
          ['منصوب','کرد'],['منصوب','شد'],['منصوب','شدند'],['انتصاب'],['برکنار','شد'],['برکنار','کرد'],['اخراج','شد'],['اخراج','کرد'],
          ['سرسام','آوری'],['سرسام','آور'],['معرفی','شد'],['معرفی','کرد'],['تحسین','برانگیزی'],['تحسین','برانگیز'],['معرفی','نسل','جدید'],
          ['بهترین','سال'],['اهدای','جوایز'],['اهدا','کرد'],['پشتیبانی','میکند'],['پشتیبانی','می‌کند'],['پشتیبانی','میکنند'],['پشتیبانی','می‌کنند'],
          ['نسخه','فروش'],['رکورد','فروش'],['میلیون','کاربر'],['میلیون','فروش'],['میلیارد','فروش'],['به‌روزرسانی'],['پشتیبانی','کرد'],
          ['منتشر','شد'],['منتشر','کرد'],['منتشر','میشود'],['منتشر','می‌شود'],['پرفروش‌ترین‌'],['افزایش','قیمت'],['افزایش','تقاضا'],
          ['کاهش','قیمت'],['کاهش','تقاضا'],['رسمی'],['لو','رفت'],['خرید','شرکت'],['تاریخ','انتشار'],['تعطیل','شد'],['بهترین','نسخه'],
          ['آپدیت','بزرگ'],['آپدیت','بزرگی'],['اپدیت','بزرگ'],['رونمایی'],['تایید','کرد'],['افت','سرعت','اینترنت'],['کاهش','سرعت','اینترنت'],
          ['متوقف','شد'],['متوقف','کرد'],['اعتراض'],['تاخیر','عرضه'],['رسما','اعلام'],['هزینه','میلیون'],['هزینه','میلیارد'],['تهدید','کرد'],
          ['اولین','تصاویر'],['فوت','کرد'],['فوت','شد'],['دست','ساخت'],['پیشتازی'],['پیشتاز'],['ترک','کرد'],['شایعه'],['برگزاری','رویداد'],
          ['برگزار','رویداد'],['عرضه','کرد'],['عرضه','شد'],['عرضه','می‌شود'],['عرضه','میشود'],['زمان','معرفی'],['تاریخ','معرفی'],
          ['تاخیر','خورد'],['حالت','تعلیق'],['حذف','شد'],['حذف','شدند'],['حذف','کرد'],['انتشار','بیانیه‌'],['انتشار','بیانیه‌ای'],['خرید','کمپانی'],
          ['رفع','باگ'],['مشکلات','امنیتی'],['اختلال','سرور'],['مالکیت','دست','داد'],['رفع','مشکلات'],['فاش','شد'],['فاش','کرد'],
          ['فاش','شدند'],['عذرخواهی','کرد'],['عذرخواهی','کردند'],['شایعات'],['افشای','اطلاعات'],['موفقیت','خیره‌کننده‌'],['گران','شد'],
          ['گران','کرد'],['ارزان','شد'],['ارزان','کرد'],['موفقیت','غیرمنتظره‌'],['تحریم','کرد'],['تحریم','شد'],['خرید','استودیو'],
          ['فروش','استودیو'],['تصاحب','کرد'],['میلیارد','دلار'],['میلیون','دلار'],['اولین','رسمی'],['موجب','خشم'],
          ['پایان','پشتیبانی'],['سقوط','ارزش','سهام'],['صعود','ارزش','سهام'],['افزایش','ارزش','سهام'],['کاهش','ارزش','سهام'],
          ['میلیارد','نفر'],['میلیون','نفر'],['پایان','سلطه‌'],['تاخیر','خورده']
]

### Celebrity Key-Phrases

In [ ]:
list_of_celebrity_positive_keyphrase = [
                      ['منصوب','کرد'],['منصوب','شد'],['منصوب','شدند'],['انتصاب'],['برکنار','شد'],['برکنار','کرد'],['اخراج','شد'],
                      ['اخراج','کرد'],['دریاچه','ارومیه'],['لغو','شد'],['لغو','کرد'],['تعطیل','شد'],['تعطیل','شدند'],['تعطیل','کرد'],
                      ['منتقل','بیماستان'],['انتقال','بیمارستان'],['انتقال','مصدومان'],['انتقال','مصدوم'],['انتقال','مجروحین'],['شناسایی','ضاربان'],
                      ['تسلیت'],['شهادت'],['مصوبه','کمیته'],['افزایش','شدید'],['کاهش','شدید'],['اعلام','کمیته'],['شرایط','خطرناک'],
                      ['وضعیت','خطرناک'],['تشکیل','کمیته‌'],['افشای','دلایل'],['متوقف','شد'],['بسته','شد'],['شریک','جرم'],
                      ['ثبت','رکورد'],['خطر','انقراض'],['برای','اولین','بار'],['محکوم','شد'],['بی','سابقه'],['بی‌سابقه'],['تلفات','انسانی'],
                      ['معضلی','جهانی'],['معضل','جهانی'],['غیرمجاز'],['غیر‌مجاز'],['غیر','مجاز'],['بحران'],['برگزاری','کنفرانس'],['عرضه','شد'],
                      ['رونمایی','کرد'],['رونمایی','شد'],['تشییع','پیکر'],['فوت','کرد'],['فوت','شد'],['درگذشت'],['نجات','نسل'],['هدف','گلوله'],
                      ['سلاح','جنگی'],['تقدیر','مراسم'],['تهدید','می‌کند'],['تهدید','کرد'],['تیر','خورده'],['رکورد'],['ممنوع','شد'],
                      ['معرض','خطر'],['افتتاح'],['شناسایی','شد'],['هشدار','مقام‌های'],['تعقیب','گریز'],['دستگیری','عوامل'],['هشدار','جدی'],
                      ['بازداشت','شدند'],['بازداشت','کردند'],['بازداشت','شد'],['بازداشت','کردند'],['سانحه'],['حمله','سلاح'],['کشف','شد'],
                      ['کشف','کرد'],['مهمانی','شبانه'],['جنجال','سلبریتی'],['جنجالی','سلبریتی'],['چهره','ماندگار'],['سلبریتی','پرحاشیه'],
                      ['سلبریتی','پر','حاشیه'],['کشف','حجاب'],['حواشی','مراسم'],['حواشی','شدید'],['مهمانی','جنجالی'],['دورهمی','بازیگران'],
                      ['کمیته','انضباطی'],['ممنوع','التصویر'],['ممنوع','تصویر'],['ممنوع','کار'],['ممنوع','الکار'],['مصدومیت','شدید'],['محروم','شد'],
                      ['محروم','کرد'],['کمیته','اخلاق'],['حادثه','دلخراش'],['دستگیری','مهمانی'],['بازیگر','معروف'],['فوتبالیست','معروف'],
                      ['کشف','مواد','مخدر'],['وضعیت','تاسف','بار'],['تجمع','مردم'],['تغییر','جنسیت'],['حمله','تند'],['درگیری','شدید'],['مهاجرت','کرد'],
                      ['برنده','جایزه'],['برنده','جایزه','بهترین'],['ازدحام','عکاسان'],['وایرال','شد'],['عجیب','آنتن','زنده'],['جنجال','برنامه','زنده'],
                      ['جنجال','آنتن','زنده'],['حرکت','عجیب','معروف'],['عصبانیت','شدید'],['مورد','توجه','عکاسان'],['استقبال','شدید'],
                      ['استقبال','چشمگیر'],['ازدواج','کرد'],['شایعه'],['شایعات'],['طلاق','گرفت'],['گریه','بی','امان'],['سوال','جنجالی'],['مهاجرت','کرد'],
                      ['پناهنده','شد'],['توقیف','شد'],['توقیف','کرد'],['توضیح','حواشی','اخیر'],['تشویق','شدید','حضار'],['حمله','شدید'],['سوگواری'],
                      ['حمله','تند'],['رونمایی','کرد'],['دریافت','جایزه'],['پوشش','منشوری'],['استوری','خبرساز'],['استوری','جنجالی'],['متاهل','شد'],
                      ['کاهش','وزن','زیاد'],['کاهش','وزن','شدید'],['افزایش','وزن','زیاد'],['افزایش','وزن','شدید'],['حرکت','باورنکردنی'],
                      ['خوش','گذرونی','لاکچری'],['زندگی','لاکچری'],['افشای','تصاویر'],['افشای','اطلاعات'],['اطلاعات','محرمانه'],['استایل','منشوری'],
                      ['لو','رفت'],['لو','رفتن'],['دسترس','خارج','شد'],['تولد','لاکچری'],['خیانت','کرد'],['سوژه','عکاسان'],
                      ['مصاحبه','جنجالی'],['حاضر','مصاحبه','نشد'],['پوشش','نامتعارف'],['دستمزد','نجومی'],['دستمزدهای','نجومی'],['هشدار','پلیس'],['فرار','مالیاتی'],
                      ['صادر','حکم'],['صادر','رای'],['انتقاد','تند'],['واکنش','عجیب'],['واکنش','جنجالی'],['پرداخت','غرامت']
]

## Sub-Functions Needed

In [ ]:
def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_url(tweet):
    return re.sub(r"http\S+", " ", tweet)

def remove_usernames(tweet):
    return re.sub('@[^\s]+',' ',tweet)

def remove_English(tweet):
  return re.sub(r'\s*[A-Za-z]+\b', ' ' , tweet).rstrip()

def remove(text):
  text = text.replace('ھ','ه')
  text = text.replace('ئ','ی')
  text = text.replace('ؤ','و')
  text = text.replace('إ','ا')
  text = text.replace('أ','ا')
  text = text.replace('َ','')
  text = text.replace('ُ','')
  text = text.replace('ِ','')
  text = text.replace('ّ','')
  text = text.replace('ء','')
  text = text.replace('ـ','')
  text = text.replace('،',' ')
  text = text.replace('،',' ')
  text = re.sub('[^\u0621-\u0628\u062A-\u063A\u0641-\u0642\u0644-\u0648\u064E-\u0651\u0655\u067E\u0686\u0698\u06A9\u06AF\u06BE\u06CC\u06F0-\u06F9 ]', ' ', text)
  return re.sub("[^0-9\u0600-\u06FF]+", " ", text).strip()

def control_numeric(text):
  text = text
  pattern_integer = r'\d+'
  pattern_float = "\d+\.\d+"
  text = re.sub(pattern_float, ' ', text)
  text = re.sub(pattern_integer, ' ', text)
  return text

def remove_nan(dataset,column):
  dataset = dataset[dataset[column].notna()]
  return dataset

def clean_tweet(tweet):
  tweet = remove(tweet)
  tweet = remove_emoji(tweet)
  tweet = remove_usernames(tweet)
  tweet = remove_url(tweet)
  tweet = remove_English(tweet)
  tweet = control_numeric(tweet)
  return Normalizer().normalize(tweet)

def tweet_to_token(tweet):
    return word_tokenize(tweet)

def remove_StopWords(text):
  return ' '.join([word for word in text.split() if word not in stop_words])

def calculate_max_length(tweet_array):
  max_length = 0
  for tweet in tweet_array:
    if max_length < len(tweet):
      max_length = len(tweet)
  return max_length

def Text2KeyPhrase(text):
  return kw_model.extract_keywords(text, keyphrase_ngram_range=(4, 7), stop_words=None)

def plot_history(history):
    
    plt.figure(figsize=(8,5),linewidth = 7, edgecolor="whitesmoke")    
    n = len(history.history['accuracy'])
    
    plt.plot(np.arange(0,n)+1,history.history['accuracy'], color='orange',marker=".")
    plt.plot(np.arange(0,n)+1,history.history['loss'],'b',marker=".")
    
    # offset both validation curves
    plt.plot(np.arange(0,n)+ 1,history.history['val_accuracy'],'r')  
    plt.plot(np.arange(0,n)+ 1,history.history['val_loss'],'g')
    
    plt.legend(['Train Acc','Train Loss','Val Acc','Val Loss'])
    plt.grid(True)
    
    # set vertical limit to 1
    plt.gca().set_ylim(0,1)

    plt.xlabel("Number of Epochs")
    plt.ylabel("Value")
    plt.suptitle("Learning Curve", size=16, y=0.927)
    plt.show()

def token2lemma(token):
  return Lemmatizer().lemmatize(token).split('#')[0]

def get_index(tupl, pos):
  return [i for i, tupl in enumerate(tupl) if tupl[-1] == pos]

def norm_token_persian(sentence):
  return word_tokenize(Normalizer().normalize(sentence))

def persian_pos(tokens):
  tagger = POSTagger(model='postagger.model')
  removed_roles = ['POSTP','CONJ','PUNC','NUM']
  part_of_speech = tagger.tag(tokens)
  remove_indices = list()
  for i in range(len(part_of_speech)):
    if part_of_speech[i][-1] in removed_roles:
      remove_indices.append(i)
  part_of_speech = [i for j, i in enumerate(part_of_speech) if j not in remove_indices]
  return part_of_speech

def persian_lemma_stem(POS):
  noun_index = get_index(POS,'N')
  # verb_index = get_index(POS,'V')
  POS = [*map(list, POS)]
  for i in noun_index:
    POS[i][0] = Stemmer().stem(POS[i][0])
  # for j in verb_index:
  #   POS[j][0] = token2lemma(POS[j][0])
  lemma_stem = [i[0] for i in POS]
  return POS, lemma_stem, ' '.join([word for word in lemma_stem])

def count_chars(text):
  return len(text)

def count_words(text):
  return len(text.split())



## DatasetLabeling Class

In [ ]:
class DatasetLabeling:
  
  def __init__(self, 
               data_path,
               positive_keyPhrase,
               negative_keyPhrase,
               spam_keyPhrase):
        self.data_path = data_path
        self.positive_keyPhrase = positive_keyPhrase
        self.negative_keyPhrase = negative_keyPhrase
        self.spam_keyPhrase = spam_keyPhrase

  def read_data(self):
    self.data = pd.read_csv(
        self.data_path,sep=',', encoding = "utf-8-sig",on_bad_lines='skip')
    self.data['text']= self.data['text'].astype(str)
    
  def clean_text(self):
    self.data['text'] = self.data['text'].apply(clean_tweet)


  def filter(self,list_of_tokens):
    for keyphrase in self.spam_keyPhrase:
      if any(token in list_of_tokens for token in keyphrase):
        return 'Spam'
    for keyphrase in self.positive_keyPhrase:
      if all(token in list_of_tokens for token in keyphrase):
        return 'Positive'
      # for keyphrase in self.negative_keyPhrase:
      #   if all(token in list_of_tokens for token in keyphrase):
      #     return 'Negative'
    return 'Negative'

  def detect_labels(self):
    list_of_labels = []
    for tweets in self.data['text']:
      token = norm_token_persian(tweets)
      list_of_labels.append(self.filter(token))
    return list_of_labels
      
  def manual_dataset_labeling(self):
    labeled_dataset = self.data.copy()
    labeled_dataset['Label'] = self.detect_labels()
    labeled_dataset.drop(labeled_dataset[labeled_dataset.Label == 'Spam'].index, inplace=True)
    labeled_dataset.loc[labeled_dataset['Label'] == 'Positive', ['Label']]= 1
    labeled_dataset.loc[labeled_dataset['Label'] == 'Negative', ['Label']]= 0
    labeled_dataset.loc[labeled_dataset['Label'] == 'No_Sense', ['Label']]= -1
    self.labeled_dataset = labeled_dataset
  
  def Main(self):
    self.read_data()
    self.clean_text()
    self.manual_dataset_labeling()


## DataProcessor Class

In [ ]:
class DataProcessor:
  
  def __init__(self, 
               Labeled_Dataset,
               Pars_Bert_Model,
               Down_Sampleing_Factor = 1.5,
               Split_Factor = 0.2,
               flag=False,
               
               ):
        self.data = Labeled_Dataset
        self.Down_Sampleing_Factor = Down_Sampleing_Factor
        self.Split_Factor = Split_Factor
        self.flag = flag
        self.Pars_Bert_Model = Pars_Bert_Model

  def down_sampeling(self):
    positive = self.data[self.data['Label']==1]
    negative = self.data[self.data['Label']==0].sample(n=int(len(positive)*self.Down_Sampleing_Factor))
    return pd.concat([positive, negative], ignore_index=True).sample(frac=1).reset_index(drop=True)
    
  def split_data(self):
    down_sampeled = self.down_sampeling()
    feature , label = down_sampeled.iloc[:,0:] , down_sampeled['Label']
    X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size=self.Split_Factor, shuffle=True)
    self.train_dataset, self.test_dataset = X_train, X_test
    self.train_text = list(map(remove_StopWords, X_train['text']))
    self.test_text = list(map(remove_StopWords, X_test['text']))
    self.y_train = np.asarray(y_train).astype('int')
    self.y_test = np.asarray(y_test).astype('int')

  def text2sequence(self):
    tokenizer = text.Tokenizer()
    tokenizer.fit_on_texts(list(self.train_text))
    train_input = tokenizer.texts_to_sequences(self.train_text)
    self.X_train = sequence.pad_sequences(train_input, maxlen = calculate_max_length(self.train_text))

    test_input = tokenizer.texts_to_sequences(self.test_text)
    self.X_test = sequence.pad_sequences(test_input, calculate_max_length(self.X_train)) 

  def text_summarization(self):
    self.X_train_summarized = []
    self.Y_train_summarized = []
    self.flag = True
    i=0
    for text in self.train_text:
      temp = Text2KeyPhrase(text,20,20)
      if len(temp) != 0 :
        self.X_train_summarized.append(temp[0][0])
        self.Y_train_summarized.append(self.y_train[i])
      i += 1
    
  def Bert_Embedding(self):
    if not self.flag:
      self.train_embedding = self.Pars_Bert_Model.encode(self.train_text)
      self.test_embedding = self.Pars_Bert_Model.encode(self.test_text)
    else:
      self.train_embedding = self.Pars_Bert_Model.encode(self.X_train_summarized)
      self.train_embedding = self.Pars_Bert_Model.encode(self.train_text)

  
  def Main(self):
    self.split_data()
    # self.text2sequence()
    # self.text_summarization()
    # self.Bert_Embedding()

## Model Class

In [ ]:
class Model:

  def __init__(self, 
               X_train,
               y_train,
               X_test,
               y_test,
               train_embedding,
               test_embedding,
               train_dataset = [],
               test_dataset = [],
               batch_size = 128,
               n_epochs = 100,
               filters = 250,               
               kernel_size = 3,
               units = 250,
               optimizer = tf.optimizers.SGD,
               drop_rate = 0.4,
               learning_rate = 0.02,
               validation_split = 0.10
               ):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.train_embedding = train_embedding
        self.test_embedding = test_embedding
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.filters = filters
        self.kernel_size = kernel_size
        self.units = units
        self.optimizer = optimizer
        self.drop_rate = drop_rate
        self.learning_rate = learning_rate
        self.validation_split = validation_split

  def define_cnn_model(self):
      
      model = Sequential()
      
      #create embedding layer
      model.add(Embedding(self.train_embedding.shape[0], self.train_embedding.shape[1], trainable = False
                        ,embeddings_initializer = tf.keras.initializers.Constant(self.train_embedding)))    
      # 1st dropout
      model.add(Dropout(self.drop_rate))
      
      # 1st convolutional 1-D layer
      model.add(Conv1D(self.filters, self.kernel_size, padding = 'valid')) #no padding
      
      #max pooling layer
      model.add(MaxPooling1D())

      # 2nd convolutional 1-D layer
      model.add(Conv1D(self.filters, self.kernel_size, padding = 'valid')) #no padding
      
      #max pooling layer
      model.add(MaxPooling1D())
      
      # 3rd convolutional 1-D layer
      model.add(Conv1D(self.filters, self.kernel_size, padding = 'valid', activation = 'relu'))
      
      # global max pooling layer
      model.add(GlobalAveragePooling1D())
      
      # 1st dense layer
      model.add(Dense(self.units,activation = 'relu'))
      
      # 2nd dropout
      model.add(Dropout(self.drop_rate))
      
      # final dense layer
      model.add(Dense(1,activation = 'sigmoid'))
      
      # compile the model
      model.compile(loss = 'binary_crossentropy',    #since we are doing binary classification
                  optimizer = tf.optimizers.SGD(learning_rate = self.learning_rate),
                  metrics = ['accuracy']) 

      self.cnn_model = model
    
  def train_cnn(self):
    earlyStopping = EarlyStopping(monitor='val_loss', patience=25, verbose=0, mode='min')
    # mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    # reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta=1e-3, mode='min')

    self.cnn_history = self.cnn_model.fit(
        self.X_train, self.y_train,
        batch_size = self.batch_size,
        validation_split = self.validation_split,
        callbacks = [earlyStopping],
        epochs = self.n_epochs
        )
    
  def plot_cnn_training_history(self):
    return plot_history(self.cnn_history)

  def Evaluate_cnn(self):
    self.cnn_prediction = self.cnn_model.predict(self.X_test, verbose = 1, batch_size = self.batch_size)
    prediction = self.cnn_prediction
    prediction[prediction >= 0.5] = 1
    prediction[prediction < 0.5] = 0
    print(classification_report(self.y_test, prediction))
    print(confusion_matrix(self.y_test, prediction))
      
  def save_model(self, model, path):
    return pickle.dump(self.model, open(path, 'wb'))

  def load_model(path):
    return pickle.load(open(path, 'rb'))

  def svm_model(self):
    self.svm_model = svm.NuSVC(kernel='poly',gamma="auto", probability=False)
    self.svm_model.fit(self.train_embedding, self.y_train)
  
  def Evaluate_svm(self):
    self.svm_prediction = self.svm_model.predict(self.test_embedding)
    print(classification_report(self.y_test, self.svm_prediction))
    print(confusion_matrix(self.y_test, self.svm_prediction))

  def naive_bayes_model(self):
    nb_train_input = self.train_embedding + abs(np.amin(self.train_embedding))
    self.nb_model = MultinomialNB()
    self.nb_model.fit(nb_train_input, self.y_train)
  
  def Evaluate_naive_bayes(self):
    nb_test_input = self.test_embedding + abs(np.amin(self.test_embedding))
    self.nb_prediction = self.nb_model.predict(nb_test_input)
    print(classification_report(self.y_test, self.nb_prediction))
    print(confusion_matrix(self.y_test, self.nb_prediction))
  
  def feature_extraction(self):
    scaler = StandardScaler()
    self.train_dataset['svm_log_prob_0'] = pd.Series(self.svm_model.predict_proba(self.train_embedding)[::,0])
    self.train_dataset['svm_log_prob_1'] = pd.Series(self.svm_model.predict_proba(self.train_embedding)[::,-1])
    self.train_dataset['cnn_prob'] = pd.Series(self.cnn_model.predict(self.X_train, verbose = 1, batch_size = self.batch_size).T.ravel())
    self.train_dataset['num_char'] = self.train_dataset['text'].apply(count_chars)
    self.train_dataset['num_words'] = self.train_dataset['text'].apply(count_words)
    self.train_dataset.fillna(value=0, inplace=True)
    self.train_features = self.train_dataset[['svm_log_prob_0','svm_log_prob_1','num_words','num_words','like_count','retweet_count']].to_numpy()
    self.train_features = scaler.fit_transform(self.train_features)
    

    self.test_dataset['svm_log_prob_0'] = pd.Series(self.svm_model.predict_proba(self.test_embedding)[::,0])
    self.test_dataset['svm_log_prob_1'] = pd.Series(self.svm_model.predict_proba(self.test_embedding)[::,-1])
    self.test_dataset['cnn_prob'] = pd.Series(self.cnn_model.predict(self.X_test, verbose = 1, batch_size = self.batch_size).T.ravel())
    self.test_dataset['num_char'] = self.test_dataset['text'].apply(count_chars)
    self.test_dataset['num_words'] = self.test_dataset['text'].apply(count_words)
    self.test_dataset.fillna(value=0, inplace=True)
    self.test_features = self.test_dataset[['svm_log_prob_0','svm_log_prob_1','num_words','num_words','like_count','retweet_count']].to_numpy()
    self.test_features = scaler.fit_transform(self.test_features)

  def final_classifier(self):
    self.final_DT = DecisionTreeClassifier().fit(self.train_features, self.y_train)
    self.final_DT_prediction = self.final_DT.predict(self.test_features)
    print(classification_report(self.y_test, self.final_DT_prediction))
    print(confusion_matrix(self.y_test, self.final_DT_prediction))


  def Main(self):
    # print('----------- Training of SVM Model Starts -----------')
    # print()
    # self.svm_model()
    # print()
    # print('----------- Evaluation of SVM Model Starts -----------')
    # print()
    # self.Evaluate_svm()
    # print()
    # print('----------- Training of NB Model Starts -----------')
    # print()
    # self.naive_bayes_model()
    # print()
    # print('----------- Evaluation of NB Model Starts -----------')
    # print()
    # self.Evaluate_naive_bayes()
    self.define_cnn_model()
    print()
    print('----------- Training of CNN Model Starts -----------')
    print()    
    self.train_cnn()
    self.plot_cnn_training_history()
    print()
    print('----------- Evaluation of CNN Model Starts -----------')
    print()
    self.Evaluate_cnn()
    # self.feature_extraction()
    # self.final_classifier()
    # self.save_model()

## Predict Class

In [ ]:
class Predict:
  
  def __init__(self,
               
               SVM_model_tech,
               CNN_model_tech,
               tech_train_text,

               SVM_model_sport,
               CNN_model_sport,
               sport_train_text,

               SVM_model_economy,
               CNN_model_economy,
               economy_train_text,

               SVM_model_art,
               CNN_model_art,
               art_train_text,

               ParsBert
               ):
        self.SVM_model_tech = SVM_model_tech
        self.CNN_model_tech = CNN_model_tech
        self.tech_train_text = tech_train_text

        self.SVM_model_sport = SVM_model_sport
        self.CNN_model_sport = CNN_model_sport
        self.sport_train_text = sport_train_text

        self.SVM_model_economy = SVM_model_economy
        self.CNN_model_economy = CNN_model_economy
        self.economy_train_text = economy_train_text

        self.SVM_model_art = SVM_model_art
        self.CNN_model_art = CNN_model_art
        self.art_train_text = art_train_text

        self.ParsBert = ParsBert
  
  def feature_extrction(self, pp_text, category):
    text_embedding = self.ParsBert.encode(pp_text)
    tokenizer = text.Tokenizer()
    if category == 'sport':
      tokenizer.fit_on_texts(list(self.sport_train_text))
    elif category == 'tech':
      tokenizer.fit_on_texts(list(self.tech_train_text))
    elif category == 'economy':
      tokenizer.fit_on_texts(list(self.economy_train_text))
    elif category == 'art':
      tokenizer.fit_on_texts(list(self.art_train_text))

    text_input = tokenizer.texts_to_sequences([pp_text])
    encoded_text = sequence.pad_sequences(text_input, maxlen = len(pp_text))
    return text_embedding, encoded_text

  def Main(self):
    while True:
      sample = input('Please Enter your Text (Or Enter Quit): ')
      if sample == 'Quit':
        break
      category = input('Enter Category of Text (sport/ tech/ economy/ art/): ')
      
      start_time = time.time()
      sample = clean_tweet(sample)
      text_embedding, encoded_text = self.feature_extrction(sample,category)

      if category == 'sport':
        cnn_prediction = self.CNN_model_sport.predict(encoded_text, verbose = 1, batch_size = 64)
        svm_prediction = self.SVM_model_sport.predict(text_embedding.reshape(-1, 768))
        end_time = time.time()
        print('input text: ',clean_tweet(sample))
        print('CNN model output: ', cnn_prediction)
        print('SVM model output: ', svm_prediction)
        print('Time Duration:',(end_time - start_time),'Seconds')
      
      elif category == 'tech':
        cnn_prediction = self.CNN_model_tech.predict(encoded_text, verbose = 1, batch_size = 128)
        svm_prediction = self.SVM_model_tech.predict(text_embedding.reshape(-1, 768))
        end_time = time.time()
        print('input text: ',clean_tweet(sample))
        print('CNN model output: ', cnn_prediction)
        print('SVM model output: ', svm_prediction)
        print('Time Duration:',(end_time - start_time),'Seconds')
      
      elif category == 'economy':
        cnn_prediction = self.CNN_model_economy.predict(encoded_text, verbose = 1, batch_size = 128)
        svm_prediction = self.SVM_model_economy.predict(text_embedding.reshape(-1, 768))
        end_time = time.time()
        print('input text: ',clean_tweet(sample))
        print('CNN model output: ', cnn_prediction)
        print('SVM model output: ', svm_prediction)
        print('Time Duration:',(end_time - start_time),'Seconds')
      
      elif category == 'art':
        cnn_prediction = self.CNN_model_art.predict(encoded_text, verbose = 1, batch_size = 128)
        svm_prediction = self.SVM_model_art.predict(text_embedding.reshape(-1, 768))
        end_time = time.time()
        print('input text: ',clean_tweet(sample))
        print('CNN model output: ', cnn_prediction)
        print('SVM model output: ', svm_prediction)
        print('Time Duration:',(end_time - start_time),'Seconds')
 

## Thechnology Event Detection

In [ ]:
Tech_dataset = DatasetLabeling(
               '/content/gdrive/MyDrive/Event Detection Final Dataset/Technology.csv',list_of_technology_positive_keyphrase,[],spam_keyphrase)
Tech_dataset.Main()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.


In [ ]:
Tech_processed = DataProcessor(Tech_dataset.labeled_dataset,
                               Pars_Bert_Model,
                               Down_Sampleing_Factor=1.5,
                               Split_Factor=0.2)
Tech_processed.Main()
del Tech_dataset

## Sport Event Detection

In [ ]:
Sport_dataset = DatasetLabeling(
               '/content/gdrive/MyDrive/Event Detection Final Dataset/Sport.csv',list_of_sport_positive_keyphrase,[],spam_keyphrase)
Sport_dataset.Main()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.


In [ ]:
Sport_processed = DataProcessor(Sport_dataset.labeled_dataset,
                                Pars_Bert_Model,
                                Down_Sampleing_Factor=1.5,
                                Split_Factor=0.2)
Sport_processed.Main()
del Sport_dataset

## Economy Event Detection

In [ ]:
Economy_dataset = DatasetLabeling(
               '/content/gdrive/MyDrive/Event Detection Final Dataset/Economy.csv',list_of_economy_positive_keyphrase,[],spam_keyphrase)
Economy_dataset.Main()

In [ ]:
Economy_processed = DataProcessor(Economy_dataset.labeled_dataset,
                                  Pars_Bert_Model,
                                  Down_Sampleing_Factor=1.5,
                                  Split_Factor=0.2)
Economy_processed.Main()
del Economy_dataset

## Art Event Detection

In [ ]:
Art_dataset = DatasetLabeling(
               '/content/gdrive/MyDrive/Event Detection Final Dataset/Art.csv',list_of_art_positive_keyphrase,[],spam_keyphrase)
Art_dataset.Main()

In [ ]:
Art_processed = DataProcessor(Art_dataset.labeled_dataset,
                              Pars_Bert_Model,
                              Down_Sampleing_Factor=1.5,
                              Split_Factor=0.2)
Art_processed.Main()
del Art_dataset

## Load models

In [ ]:
from tensorflow.keras.models import model_from_json
import pickle

In [ ]:
json_file = open('/content/gdrive/MyDrive/Event_Detection_Models/Technology_cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
CNN_model_tech = model_from_json(loaded_model_json)
CNN_model_tech.load_weights("/content/gdrive/MyDrive/Event_Detection_Models/Technology_cnn_model.h5")
SVM_model_tech = pickle.load(open('/content/gdrive/MyDrive/Event_Detection_Models/Technology_svm_model', 'rb'))
print("Model Loaded")

In [35]:
json_file = open('/content/gdrive/MyDrive/Event_Detection_Models/Sport_cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
CNN_model_sport = model_from_json(loaded_model_json)
CNN_model_sport.load_weights("/content/gdrive/MyDrive/Event_Detection_Models/Sport_cnn_model.h5")
SVM_model_sport = pickle.load(open('/content/gdrive/MyDrive/Event_Detection_Models/Sport_svm_model', 'rb'))
print("Model Loaded")


Model Loaded


In [36]:
json_file = open('/content/gdrive/MyDrive/Event_Detection_Models/Economy_cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
CNN_model_economy = model_from_json(loaded_model_json)
CNN_model_economy.load_weights("/content/gdrive/MyDrive/Event_Detection_Models/Economy_cnn_model.h5")
SVM_model_economy = pickle.load(open('/content/gdrive/MyDrive/Event_Detection_Models/Economy_svm_model', 'rb'))
print("Model Loaded")


Model Loaded


In [ ]:
json_file = open('/content/gdrive/MyDrive/Event_Detection_Models/Art_cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
CNN_model_art = model_from_json(loaded_model_json)
CNN_model_art.load_weights("/content/gdrive/MyDrive/Event_Detection_Models/Art_cnn_model.h5")
SVM_model_art = pickle.load(open('/content/gdrive/MyDrive/Event_Detection_Models/Art_svm_model', 'rb'))
print("Model Loaded")


## Real-Time Simulation

In [38]:
Real_time_Prediction = Predict(
               SVM_model_tech,
               CNN_model_tech,
               Tech_processed.train_text,

               SVM_model_sport,
               CNN_model_sport,
               Sport_processed.train_text,

               SVM_model_economy,
               CNN_model_economy,
               Economy_processed.train_text,

               SVM_model_art,
               CNN_model_art,
               Art_processed.train_text,

               Pars_Bert_Model
               )

In [39]:
Real_time_Prediction.Main()

Please Enter your Text (Or Enter Quit): سامسونگ از جدید ترین گوشی خود رونمایی کرد
Enter Category of Text (sport/ tech/ economy/ art/): tech
1/1 [==============================] - 7s 7s/step
input text:  سامسونگ از جدید‌ترین گوشی خود رونمایی کرد
CNN model output:  [[0.02772751]]
SVM model output:  [1]
Time Duration: 23.096371173858643 Seconds
Please Enter your Text (Or Enter Quit): سرور های سامانه سوخت ایران با مشکل مواجه شدند
Enter Category of Text (sport/ tech/ economy/ art/): tech
1/1 [==============================] - 0s 376ms/step
input text:  سرور‌های سامانه سوخت ایران با مشکل مواجه شدند
CNN model output:  [[0.02724305]]
SVM model output:  [0]
Time Duration: 2.6027352809906006 Seconds
Please Enter your Text (Or Enter Quit): علی دایی جایزه بهترین بازیکن سال اروپا را دریافت کرد
Enter Category of Text (sport/ tech/ economy/ art/): sport
1/1 [==============================] - 0s 131ms/step
input text:  علی دایی جایزه بهترین بازیکن سال اروپا را دریافت کرد
CNN model output:  [[1.]]
SVM 

1/1 [==============================] - 0s 230ms/step
input text:  آزمایش دوپینگ بازیکن استقلال مثبت شد و این بازیکن به کمیته انصباطی دعوت شد
CNN model output:  [[0.6380742]]
SVM model output:  [1]
Time Duration: 0.8045134544372559 Seconds
Please Enter your Text (Or Enter Quit): شاهد بازی بسیار خسته کننده ای بودیم
Enter Category of Text (sport/ tech/ economy/ art/): sport
1/1 [==============================] - 0s 60ms/step
input text:  شاهد بازی بسیار خسته کننده‌ای بودیم
CNN model output:  [[0.07941524]]
SVM model output:  [0]
Time Duration: 0.5063791275024414 Seconds
Please Enter your Text (Or Enter Quit): آمریکا هیچ غلطی نمیتواند بکند
Enter Category of Text (sport/ tech/ economy/ art/): economy


1/1 [==============================] - 0s 157ms/step
input text:  آمریکا هیچ غلطی نمیتواند بکند
CNN model output:  [[0.47550246]]
SVM model output:  [0]
Time Duration: 1.175750494003296 Seconds
Please Enter your Text (Or Enter Quit): مجوز کنسرت شادمهر عقیلی صادر نشد
Enter Category of Text (sport/ tech/ economy/ art/): art
1/1 [==============================] - 0s 153ms/step
input text:  مجوز کنسرت شادمهر عقیلی صادر نشد
CNN model output:  [[1.]]
SVM model output:  [1]
Time Duration: 1.9643993377685547 Seconds
Please Enter your Text (Or Enter Quit): اثر زیبایی کشف شد که تا به حال دیده نشده بود
Enter Category of Text (sport/ tech/ economy/ art/): art
1/1 [==============================] - 0s 110ms/step
input text:  اثر زیبایی کشف شد که تا به حال دیده نشده بود
CNN model output:  [[1.]]
SVM model output:  [0]
Time Duration: 1.9587509632110596 Seconds
Please Enter your Text (Or Enter Quit): موادی کشف شد که تا به حال دیده نشده بود
Enter Category of Text (sport/ tech/ economy/ art/): art
1/1 [